Import relevant libraries

In [33]:
#%pip install tsai
#%pip install ta
#%pip install optuna
#%pip install sklearn
#%pip install matplotlib

import pandas as pd
from Preprocessing.preprocessing import preprocessing
import time
import torch
from tsai.all import *
import os
from Optimizer.optimizer import optimize_model, optimize_data, get_binary_accuracy_clf
from Evaluation.evaluation_tsai import calculate_pseudo_return, get_plots_classification, get_plots_regression
from fastai.callback.tracker import EarlyStoppingCallback
import time

print('GPU support:', torch.cuda.is_available())
computer_setup()

GPU support: True
os              : Windows-10-10.0.19044-SP0
python          : 3.9.16
tsai            : 0.3.5
fastai          : 2.7.11
fastcore        : 1.5.28
torch           : 1.13.1
device          : 1 gpu (['NVIDIA GeForce GTX 970'])
cpu cores       : 4
threads per cpu : 1
RAM             : 15.95 GB
GPU memory      : [4.0] GB


Finds the best features

In [34]:
file = 'Data\\twelve_data\AAPL_15min'

df = pd.read_csv(file)
index = file.split('\\')[-1].split('_')[-1]
data = 'twelve' if 'twelve' in file else 'alpacca'

preprocessing_params = {
    'df': df[:3000],
    'TSAI': True,
    'CLF': False,
    'index': index,
    'data': data,
    'sequence_length' : 30
}

model_type = "mini_rocket"
folder = '100'
Classifier = preprocessing_params['CLF']
model_type = model_type + '_class' if Classifier else model_type + '_reg'

data_opti = False
if data_opti:
    optimize_data(model_type=model_type, preprocessing_params=preprocessing_params, n_trials=50, n_epochs=20, batch_size = 64, index = preprocessing_params.pop('index', None))

    def get_results(df, groupby_col):
        agg_results = df.groupby(groupby_col).agg({'bin_accuracy': ['mean', 'max', 'min', 'count']}).reset_index()
        agg_results.columns = [groupby_col, 'Mean', 'High', 'Low', 'Count']
        agg_results = agg_results.sort_values(by='Mean', ascending=False)
        return agg_results

    if folder == '100':
        df = pd.read_csv(f"Optimizer/data_optimization/{model_type}/{100}/{model_type}_{index}_100_hyperparameters_results.csv").fillna('None')
    else:
        df = pd.read_csv(f"Optimizer/data_optimization/{model_type}/{100}/{model_type}_{index}_hyperparameters_results.csv").fillna('None')
        
    groupby_cols = ['lag', 'dif_all', 'train_size', 'buckets', 'TI']
    results = {col: get_results(df, col) for col in groupby_cols}

    for col in groupby_cols:
        print(f"Results for {col}:")
        print(results[col])

Optimize the chosen model

In [35]:
from Optimizer.optimizer import optimize_model

amazon = False
if amazon:
    file = 'Data\\twelve_data\AAPL_1min'
    index = file.split('\\')[-1].split('_')[-1]
else:
    file = 'Data/twelve_data/AAPL_1min'
    index = file.split('/')[-1].split('_')[-1]

data = 'twelve' if 'twelve' in file else 'alpacca'
df = pd.read_csv(file)

preprocessing_params = {
    'df': df,
    'lag': 15,
    'dif_all': True,
    'train_size': 0.95,
    'TSAI': True,
    'CLF': True,
    'index': None,
    'data': data,
    'buckets': 10,
    'TI' : True
}

model_type = 'tst'
folder = '100'
Classifier = preprocessing_params['CLF']
model_type = model_type + '_class' if Classifier else model_type + '_reg'

opti = False
if opti:
    optimize_model(model_type=model_type, preprocessing_params=preprocessing_params, n_trials=1, n_epochs=1, folder=folder, index_time = index)

results_df = pd.read_csv(f"models/{model_type}/{folder}/{index}/{model_type}_hyperparameters_results.csv")

if Classifier: 
    df = results_df.sort_values(by='val_accuracy', ascending=False)
else: 
    df = results_df.sort_values(by='val_mae', ascending=True)

df

,model,df_len,epochs,seq_length,batch_size,learning_rate,d_model,n_layers,n_heads,d_ff,dropout,val_accuracy,time
24,tst_class,199961,20,30,128,0.0001,512,1,8,512,0.4,0.68,193.279337
19,tst_class,199961,20,30,128,0.0001,512,1,8,512,0.6,0.67,443.434187
73,tst_class,199961,20,30,128,0.0001,512,1,8,512,0.4,0.65,402.844881
67,tst_class,199961,20,30,128,0.0010,512,1,16,512,0.2,0.65,322.990533
85,tst_class,199961,20,20,128,0.0001,512,2,8,512,0.4,0.65,307.735555
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,tst_class,199961,20,10,128,0.0001,512,4,8,512,0.4,0.41,575.344904
45,tst_class,199961,20,30,128,0.0001,512,1,8,512,0.4,0.39,196.971211
7,tst_class,199961,20,40,64,0.0010,64,8,32,256,0.2,0.37,681.632344
93,tst_class,199961,20,30,128,0.0001,512,8,8,512,0.4,0.32,2024.184410


Choose model and its optimal data and hyperparameter and preprocesses it and trains the model

In [69]:
test_sets_dir = 'Data/test_sets/1min/'

# Get a list of all files in the directory
files = os.listdir(test_sets_dir)

# Loop through each file in the directory
for file in files:
    # Load the file into a pandas dataframe
    path = os.path.join(test_sets_dir, file)
    print(path)
    df = pd.read_csv(path)


Data/test_sets/1min/BTCUSD_1min
Data/test_sets/1min/CHGG_1min
Data/test_sets/1min/CRM_1min
Data/test_sets/1min/ETHUSD_1min
Data/test_sets/1min/GLD_1min
Data/test_sets/1min/LOGI_1min
Data/test_sets/1min/UNG_1min
Data/test_sets/1min/USDTUSD_1min
Data/test_sets/1min/USO_1min


,datetime,open,high,low,close,volume
0,2021-02-12 15:38:00,39.885,39.895,39.8800,39.895,6250
1,2021-02-12 15:39:00,39.890,39.895,39.8900,39.890,6603
2,2021-02-12 15:40:00,39.890,39.895,39.8850,39.885,4460
3,2021-02-12 15:41:00,39.885,39.905,39.8850,39.900,7420
4,2021-02-12 15:42:00,39.910,39.915,39.9100,39.915,7109
...,...,...,...,...,...,...
194957,2023-03-01 15:55:00,68.040,68.040,68.0300,68.040,17596
194958,2023-03-01 15:56:00,68.035,68.045,68.0301,68.040,7135
194959,2023-03-01 15:57:00,68.050,68.060,68.0400,68.045,39688
194960,2023-03-01 15:58:00,68.040,68.050,68.0200,68.050,26084


In [70]:
seed = 1

test_sets_dir = 'Data/test_sets/5min/'

# Get a list of all files in the directory
files = os.listdir(test_sets_dir)

# Loop through each file in the directory
for file in files:
    #Load the file into a pandas dataframe
    path = os.path.join(test_sets_dir, file)

    df = pd.read_csv(path)
    stock = file.split('/')[-1].split('_')[-2]

    trials = 10
    for _ in range(trials):

        # Set the random seed for reproducibility
        seed += 1
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)

        model_type = 'lstm_class'

        preprocessing_params = {
            'df': df,
            'lag': 1,
            'dif_all': False,
            'train_size': 0.95, #Will be adjusted in code
            'TSAI': True, #Always true
            'CLF': True, #Always true
            'index': None,
            'data': 'twelve',
            'buckets': 3,
            'TI' : True
        }

        model_params = {
            'seq_length':50, 
            'batch_size':128, 
            'learning_rate':0.01, 
            'hidden_size':100, 
            'n_layers':8, 
            'rnn_dropout':0.6, 
            'fc_dropout':0.6
        } #Adjust based on model, should be a dict

        seq_length = model_params.pop('seq_length', None)
        batch_size = model_params.pop('batch_size', None)
        learning_rate = model_params.pop('learning_rate', None)

        data_train,data_test, _ = preprocessing(**preprocessing_params, sequence_length=seq_length, print_info=False)

        # Changes the data into features and labels with the split used later in TSAI for modelling
        test_points = 100
        X, y, splits = combine_split_data([data_train[0], data_test[0][:test_points]],[data_train[1], data_test[1][:test_points]])

        # Utilizes the GPU if possible
        if torch.cuda.is_available(): X, y = X.cuda(), y.cuda()

        dsets = TSDatasets(X, y, splits=splits)
        dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=batch_size)

        nr_features = X.shape[1] # Number of features
        nr_labels = torch.unique(y).numel() if Classifier else 1 # Number of labels

        # Initiates the models
        if model_type == 'lstm_class' or model_type == 'lstm_reg': model = LSTMPlus(c_in=nr_features, c_out=nr_labels, **model_params)
        if model_type == 'lstm_fcn_class' or model_type == 'lstm_fcn_reg': model = LSTM_FCNPlus(c_in=nr_features, c_out=nr_labels, **model_params, shuffle=False)
        if model_type == 'tst_class' or model_type == 'tst_reg': model = TST(c_in=nr_features, c_out=nr_labels, seq_len=seq_length, **model_params)
        #if model_type == 'mini_rocket': model = 

        learner = Learner(dls, model, loss_func=LabelSmoothingCrossEntropyFlat(), metrics=accuracy, cbs=[EarlyStoppingCallback(patience=5)])

        # Trains the model
        epochs = 20
        start = time.time()
        learner.fit_one_cycle(n_epoch = epochs, lr_max=learning_rate)
        training_time = time.time() - start

        acc = get_binary_accuracy_clf(learner, X[splits[1]], y[splits[1]], preprocessing_params['buckets'])

        # Add the results to the pandas dataframe
        results_df = pd.read_csv('Results/prediction_results.csv') if os.path.exists('Results/prediction_results.csv') else pd.DataFrame(columns=['Model', 'Stock', 'Accuracy', 'Time'])
        results_df = results_df.append({
            'Model': model_type,
            'Stock': stock,
            'Accuracy': acc,
            'Time': training_time
        }, ignore_index=True)

        # Save the dataframe to a csv file
        results_df.to_csv('Results/prediction_results.csv', index=False)

        print('Stock:,', stock, ' Accuracy:', acc)

Data/test_sets/5min/BTCUSD_5min


Stock:, BTCUSD  Accuracy: 0.5


Stock:, BTCUSD  Accuracy: 0.53
Data/test_sets/5min/CHGG_5min


Stock:, CHGG  Accuracy: 0.48


Stock:, CHGG  Accuracy: 0.48
Data/test_sets/5min/CRM_5min


KeyboardInterrupt: 

In [ ]:
#if torch.cuda.isavailable(): X, y = X.cpu(), y.cpu()

#get_plots_classification(learner = learner, X = X, y = y, splits = splits, buckets = preprocessing_params["buckets"])